# dice_fsharp (Polyglot)

In [ ]:
#!import ../../../lib/fsharp/Testing.dib

In [ ]:
#!import ../../../lib/fsharp/Common.fs

In [ ]:
open Common

## sixthPowerSequence

In [ ]:
let sixthPowerSequence = 1 |> Seq.unfold (fun state -> Some (state, state * 6)) |> Seq.cache

In [ ]:
//// test

sixthPowerSequence
|> Seq.take 8
|> Seq.toList
|> _equal [ 1; 6; 36; 216; 1296; 7776; 46656; 279936 ]

[ 1, 6, 36, 216, 1296, 7776, 46656, 279936 ]


## accumulateDiceRolls

In [ ]:
let rec accumulateDiceRolls log rolls power acc =
    match rolls with
    | _ when power < 0 ->
        log |> Option.iter ((|>) $"accumulateDiceRolls / power: {power} / acc: {acc}")
        Some (acc + 1, rolls)
    | [] -> None
    | roll :: rest when roll > 1 ->
        let coeff = sixthPowerSequence |> Seq.item power
        let value = (roll - 1) * coeff
        log |> Option.iter ((|>) $"accumulateDiceRolls / \
            power: {power} / acc: {acc} / roll: {roll} / value: {value}"
        )
        accumulateDiceRolls log rest (power - 1) (acc + value)
    | roll :: rest ->
        log |> Option.iter ((|>) $"accumulateDiceRolls / power: {power} / acc: {acc} / roll: {roll}")
        accumulateDiceRolls log rest (power - 1) acc

In [ ]:
//// test

accumulateDiceRolls (Some (printfn "%s")) [ 6; 5; 4; 3; 2 ] 0 1000
|> _equal (Some (1006, [ 5; 4; 3; 2 ]))

accumulateDiceRolls / power: 0 / acc: 1000 / roll: 6 / value: 5
accumulateDiceRolls / power: -1 / acc: 1005
FSharpOption<Tuple<Int32,FSharpList<Int32>>>
      Value:       - 1006
      - [ 5, 4, 3, 2 ]


In [ ]:
//// test

accumulateDiceRolls (Some (printfn "%s")) [ 6; 5; 4; 3; 2 ] 1 1000
|> _equal (Some (1035, [ 4; 3; 2 ]))

accumulateDiceRolls / power: 1 / acc: 1000 / roll: 6 / value: 30
accumulateDiceRolls / power: 0 / acc: 1030 / roll: 5 / value: 4
accumulateDiceRolls / power: -1 / acc: 1034
FSharpOption<Tuple<Int32,FSharpList<Int32>>>
      Value:       - 1035
      - [ 4, 3, 2 ]


In [ ]:
//// test

accumulateDiceRolls (Some (printfn "%s")) [ 6; 5; 4; 3; 2 ] 2 1000
|> _equal (Some (1208, [ 3; 2 ]))

accumulateDiceRolls / power: 2 / acc: 1000 / roll: 6 / value: 180
accumulateDiceRolls / power: 1 / acc: 1180 / roll: 5 / value: 24
accumulateDiceRolls / power: 0 / acc: 1204 / roll: 4 / value: 3
accumulateDiceRolls / power: -1 / acc: 1207
FSharpOption<Tuple<Int32,FSharpList<Int32>>>
      Value:       - 1208
      - [ 3, 2 ]


## rollWithinBounds

In [ ]:
let rollWithinBounds log max rolls =
    let power = List.length rolls - 1
    match accumulateDiceRolls log rolls power 0 with
    | Some (result, _) when result >= 1 && result <= max -> Some result
    | _ -> None

In [ ]:
//// test

rollWithinBounds (Some (printfn "%s")) 2000 [ 1; 5; 4; 4; 5 ]
|> _equal (Some 995)

accumulateDiceRolls / power: 4 / acc: 0 / roll: 1
accumulateDiceRolls / power: 3 / acc: 0 / roll: 5 / value: 864
accumulateDiceRolls / power: 2 / acc: 864 / roll: 4 / value: 108
accumulateDiceRolls / power: 1 / acc: 972 / roll: 4 / value: 18
accumulateDiceRolls / power: 0 / acc: 990 / roll: 5 / value: 4
accumulateDiceRolls / power: -1 / acc: 994
FSharpOption<Int32>
      Value: 995


In [ ]:
//// test

rollWithinBounds (Some (printfn "%s")) 2000 [ 2; 2; 6; 4; 5 ]
|> _equal (Some 1715)

accumulateDiceRolls / power: 4 / acc: 0 / roll: 2 / value: 1296
accumulateDiceRolls / power: 3 / acc: 1296 / roll: 2 / value: 216
accumulateDiceRolls / power: 2 / acc: 1512 / roll: 6 / value: 180
accumulateDiceRolls / power: 1 / acc: 1692 / roll: 4 / value: 18
accumulateDiceRolls / power: 0 / acc: 1710 / roll: 5 / value: 4
accumulateDiceRolls / power: -1 / acc: 1714
FSharpOption<Int32>
      Value: 1715


In [ ]:
//// test

rollWithinBounds (Some (printfn "%s")) 2000 [ 4; 1; 1; 2; 3 ]
|> _equal None

accumulateDiceRolls / power: 4 / acc: 0 / roll: 4 / value: 3888
accumulateDiceRolls / power: 3 / acc: 3888 / roll: 1
accumulateDiceRolls / power: 2 / acc: 3888 / roll: 1
accumulateDiceRolls / power: 1 / acc: 3888 / roll: 2 / value: 6
accumulateDiceRolls / power: 0 / acc: 3894 / roll: 3 / value: 2
accumulateDiceRolls / power: -1 / acc: 3896
<null>


## calculateDiceCount

In [ ]:
let inline calculateDiceCount log max =
    let rec loop n p =
        if p < max
        then loop (n + 1) (p * 6)
        else
            log |> Option.iter ((|>) $"calculateDiceCount / max: {max} / n: {n} / p: {p}")
            n
    if max = 1
    then 1
    else loop 0 1

In [ ]:
//// test

calculateDiceCount (Some (printfn "%s")) 36
|> _equal 2

calculateDiceCount / max: 36 / n: 2 / p: 36
2


In [ ]:
//// test

calculateDiceCount (Some (printfn "%s")) 7777
|> _equal 6

calculateDiceCount / max: 7777 / n: 6 / p: 46656
6


## rollDice

In [ ]:
#if FABLE_COMPILER_RUST
let rollDice () : int =
#if !WASM
    Fable.Core.RustInterop.emitRustExpr () "rand::Rng::gen_range(&mut rand::thread_rng(), 1..7)"
#else
    1
#endif
#else
let private random = System.Random ()
let rollDice () =
    random.Next (1, 7)
#endif

## rotateNumber

In [ ]:
let rotateNumber max n =
    (n - 1 + max) % max + 1

## rotateNumbers

In [ ]:
let rotateNumbers max items =
    items |> Seq.map (rotateNumber max)

In [ ]:
//// test

[ -1 .. 14 ]
|> rotateNumbers 6
|> Seq.toList
|> _equal [ 5; 6; 1; 2; 3; 4; 5; 6; 1; 2; 3; 4; 5; 6; 1; 2 ]

[ 5, 6, 1, 2, 3, 4, 5, 6, 1, 2, 3, 4, 5, 6, 1, 2 ]


## createSequentialRoller

In [ ]:
let createSequentialRoller list =
    let mutable currentIndex = 0
    fun () ->
        match list |> List.tryItem currentIndex with
        | Some item ->
            currentIndex <- currentIndex + 1
            item
        | None ->
            failwith "createSequentialRoller / End of list"

## rollProgressively

In [ ]:
let rollProgressively log roll reroll max =
    let power = (calculateDiceCount log max) - 1
    let rec loop rolls size =
        if size < power + 1
        then loop (roll () :: rolls) (size + 1)
        else
            match accumulateDiceRolls log rolls power 0 with
            | Some (result, _) when result <= max -> result
            | _ when reroll -> loop (List.init power (fun _ -> roll ())) power
            | _ -> loop (roll () :: rolls) (size + 1)
    loop [] 0

In [ ]:
//// test

rollProgressively None rollDice false 1
|> _equal 1

1


In [ ]:
//// test

let sequentialRoll = createSequentialRoller [ 5; 4; 4; 5; 1 ]

rollProgressively (Some (printfn "%s")) sequentialRoll false 2000
|> _equal 995

calculateDiceCount / max: 2000 / n: 5 / p: 7776
accumulateDiceRolls / power: 4 / acc: 0 / roll: 1
accumulateDiceRolls / power: 3 / acc: 0 / roll: 5 / value: 864
accumulateDiceRolls / power: 2 / acc: 864 / roll: 4 / value: 108
accumulateDiceRolls / power: 1 / acc: 972 / roll: 4 / value: 18
accumulateDiceRolls / power: 0 / acc: 990 / roll: 5 / value: 4
accumulateDiceRolls / power: -1 / acc: 994
995


In [ ]:
//// test

let sequentialRoll = createSequentialRoller [ 5; 4; 4; 5; 2 ]

fun () -> rollProgressively (Some (printfn "%s")) sequentialRoll false 2000 |> ignore
|> _throwsC (fun ex _ ->
    printException ex
    |> _equal "System.Exception: createSequentialRoller / End of list"
)

FSI_0031+it@5-11
calculateDiceCount / max: 2000 / n: 5 / p: 7776
accumulateDiceRolls / power: 4 / acc: 0 / roll: 2 / value: 1296
accumulateDiceRolls / power: 3 / acc: 1296 / roll: 5 / value: 864
accumulateDiceRolls / power: 2 / acc: 2160 / roll: 4 / value: 108
accumulateDiceRolls / power: 1 / acc: 2268 / roll: 4 / value: 18
accumulateDiceRolls / power: 0 / acc: 2286 / roll: 5 / value: 4
accumulateDiceRolls / power: -1 / acc: 2290
System.Exception: createSequentialRoller / End of list


In [ ]:
//// test

[ 1 .. 100 ]
|> List.iter (fun n ->
    [ 0 .. 1 ]
    |> List.iter (fun reroll ->
        [ 1 .. 3500 ]
        |> List.map (fun _ -> rollProgressively None rollDice (reroll = 1) n)
        |> List.groupBy id
        |> List.length
        |> __equal false n
    )
)

## main

In [ ]:
let main args =
    let result = rollProgressively (Some (printfn "%s")) rollDice true (System.Int32.MaxValue / 10)
    trace Debug (fun () -> $"main / result: {result}") getLocals
    0